Looking at modeling equations; and using pytorch + fastai for general numerical tasks.

WNixalo – 2018/6/25 (WiP)

In [1]:
from collections import defaultdict
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [2]:
def Force(m,a):
    return m*a

def mass(F,a):
    if a == 0: return np.inf
    return F/a

def accel(F,m):
    if F == 0: return np.inf
    return m/F

In [3]:
fns = {'F':Force, 'm':mass, 'a':accel}
vals = defaultdict(list)
for fn in fns:
    vals[fn] = [[[f'{i},{j}', fns[fn](i,j)] for j in range(20)] for i in range(20)]

In [4]:
x = []; y = []
fn_modes = ['F','m','a']
for i,f in enumerate(fn_modes):
    for val in vals[f]:
        for v in val:
            args,out = v
            args = [int(a) for a in args.split(',')]
            args = [i, *args]
            x.append(args)
            y.append(out)

In [5]:
x[1000:1010]

[[2, 10, 0],
 [2, 10, 1],
 [2, 10, 2],
 [2, 10, 3],
 [2, 10, 4],
 [2, 10, 5],
 [2, 10, 6],
 [2, 10, 7],
 [2, 10, 8],
 [2, 10, 9]]

In [6]:
y[1000:1010]

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [7]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc0 = nn.Linear(3, 20)
        self.fc1 = nn.Linear(20,1)
    def forward(self, x):
        x = F.relu(self.fc0(x))
        x = F.relu(self.fc1(x))
        return x

In [8]:
model = NN()
optimizer = torch.optim.Adam(model.parameters())
criterion = F.binary_cross_entropy

In [9]:
torch.FloatTensor(x[0])

tensor([ 0.,  0.,  0.])

In [10]:
model(torch.FloatTensor(x[0])).shape

torch.Size([1])

In [11]:
from fastai.conv_learner import *

In [12]:
class NumDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        assert len(x) == len(y)
        self.transform = None
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, i):
        return torch.FloatTensor(self.x[i]), torch.FloatTensor([self.y[i]])

In [13]:
train_dataset = NumDataset(x,y)

In [14]:
train_dataset[0]

(tensor([ 0.,  0.,  0.]), tensor([ 0.]))

In [15]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16)

In [16]:
md = ModelData.from_dls(Path(os.getcwd())/'data', train_loader, train_loader)

In [53]:
x,y = next(iter(md.trn_dl))
x,y

(tensor([[  0.,   0.,   0.],
         [  0.,   0.,   1.],
         [  0.,   0.,   2.],
         [  0.,   0.,   3.],
         [  0.,   0.,   4.],
         [  0.,   0.,   5.],
         [  0.,   0.,   6.],
         [  0.,   0.,   7.],
         [  0.,   0.,   8.],
         [  0.,   0.,   9.],
         [  0.,   0.,  10.],
         [  0.,   0.,  11.],
         [  0.,   0.,  12.],
         [  0.,   0.,  13.],
         [  0.,   0.,  14.],
         [  0.,   0.,  15.]]), tensor([[ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.]]))

In [54]:
y.shape

torch.Size([16, 1])

In [55]:
x.shape

torch.Size([16, 3])

In [56]:
learner = Learner.from_model_data(NN(), md)

In [70]:
def loss_fn(z, y):
#     if y != 0:
#         return torch.abs((z-y)/y)
#     else:
#         return torch.min(1.0, torch.abs(z-y))
    return torch.abs((z-y)/y)

In [71]:
learner.crit = loss_fn

In [72]:
learner.lr_find()
learner.sched.plot(n_skip_end=0)

  0%|          | 0/75 [00:00<?, ?it/s]


RuntimeError: grad can be implicitly created only for scalar outputs

In [59]:
learner.model(x), y

(tensor([[ 0.0496],
         [ 0.1746],
         [ 0.3986],
         [ 0.6396],
         [ 0.8646],
         [ 1.0819],
         [ 1.2992],
         [ 1.5165],
         [ 1.7338],
         [ 1.9511],
         [ 2.1684],
         [ 2.3857],
         [ 2.6030],
         [ 2.8202],
         [ 3.0375],
         [ 3.2548]]), tensor([[ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.]]))

In [60]:
learner.crit(learner.model(x), y)

tensor(1.8740)

In [61]:
learner.fit(1e-10, 1)

epoch      trn_loss   val_loss                              
    0      nan        nan       



[nan]

In [62]:
x = learner.data.val_ds.x
y = learner.data.val_ds.y
z = learner.predict()

In [63]:
learner.model(torch.Tensor([[1,3,2]]))

tensor([[ 0.]])

In [64]:
for xi,zi,yi in zip(x,z,y):
    print(f'{xi} : {zi} : {yi}')

[0, 0, 0] : [0.] : 0
[0, 0, 1] : [0.] : 0
[0, 0, 2] : [0.] : 0
[0, 0, 3] : [0.] : 0
[0, 0, 4] : [0.] : 0
[0, 0, 5] : [0.] : 0
[0, 0, 6] : [0.] : 0
[0, 0, 7] : [0.] : 0
[0, 0, 8] : [0.] : 0
[0, 0, 9] : [0.] : 0
[0, 0, 10] : [0.] : 0
[0, 0, 11] : [0.] : 0
[0, 0, 12] : [0.] : 0
[0, 0, 13] : [0.] : 0
[0, 0, 14] : [0.] : 0
[0, 0, 15] : [0.] : 0
[0, 0, 16] : [0.] : 0
[0, 0, 17] : [0.] : 0
[0, 0, 18] : [0.] : 0
[0, 0, 19] : [0.] : 0
[0, 1, 0] : [0.] : 0
[0, 1, 1] : [0.] : 1
[0, 1, 2] : [0.] : 2
[0, 1, 3] : [0.] : 3
[0, 1, 4] : [0.] : 4
[0, 1, 5] : [0.] : 5
[0, 1, 6] : [0.] : 6
[0, 1, 7] : [0.] : 7
[0, 1, 8] : [0.] : 8
[0, 1, 9] : [0.] : 9
[0, 1, 10] : [0.] : 10
[0, 1, 11] : [0.] : 11
[0, 1, 12] : [0.] : 12
[0, 1, 13] : [0.] : 13
[0, 1, 14] : [0.] : 14
[0, 1, 15] : [0.] : 15
[0, 1, 16] : [0.] : 16
[0, 1, 17] : [0.] : 17
[0, 1, 18] : [0.] : 18
[0, 1, 19] : [0.] : 19
[0, 2, 0] : [0.] : 0
[0, 2, 1] : [0.] : 2
[0, 2, 2] : [0.] : 4
[0, 2, 3] : [0.] : 6
[0, 2, 4] : [0.] : 8
[0, 2, 5] : [0.] : 10
[0,

[1, 6, 2] : [0.] : 3.0
[1, 6, 3] : [0.] : 2.0
[1, 6, 4] : [0.] : 1.5
[1, 6, 5] : [0.] : 1.2
[1, 6, 6] : [0.] : 1.0
[1, 6, 7] : [0.] : 0.8571428571428571
[1, 6, 8] : [0.] : 0.75
[1, 6, 9] : [0.] : 0.6666666666666666
[1, 6, 10] : [0.] : 0.6
[1, 6, 11] : [0.] : 0.5454545454545454
[1, 6, 12] : [0.] : 0.5
[1, 6, 13] : [0.] : 0.46153846153846156
[1, 6, 14] : [0.] : 0.42857142857142855
[1, 6, 15] : [0.] : 0.4
[1, 6, 16] : [0.] : 0.375
[1, 6, 17] : [0.] : 0.35294117647058826
[1, 6, 18] : [0.] : 0.3333333333333333
[1, 6, 19] : [0.] : 0.3157894736842105
[1, 7, 0] : [0.] : inf
[1, 7, 1] : [0.] : 7.0
[1, 7, 2] : [0.] : 3.5
[1, 7, 3] : [0.] : 2.3333333333333335
[1, 7, 4] : [0.] : 1.75
[1, 7, 5] : [0.] : 1.4
[1, 7, 6] : [0.] : 1.1666666666666667
[1, 7, 7] : [0.] : 1.0
[1, 7, 8] : [0.] : 0.875
[1, 7, 9] : [0.] : 0.7777777777777778
[1, 7, 10] : [0.] : 0.7
[1, 7, 11] : [0.] : 0.6363636363636364
[1, 7, 12] : [0.] : 0.5833333333333334
[1, 7, 13] : [0.] : 0.5384615384615384
[1, 7, 14] : [0.] : 0.5
[1, 7, 

[2, 15, 19] : [0.] : 1.2666666666666666
[2, 16, 0] : [0.] : 0.0
[2, 16, 1] : [0.] : 0.0625
[2, 16, 2] : [0.] : 0.125
[2, 16, 3] : [0.] : 0.1875
[2, 16, 4] : [0.] : 0.25
[2, 16, 5] : [0.] : 0.3125
[2, 16, 6] : [0.] : 0.375
[2, 16, 7] : [0.] : 0.4375
[2, 16, 8] : [0.] : 0.5
[2, 16, 9] : [0.] : 0.5625
[2, 16, 10] : [0.] : 0.625
[2, 16, 11] : [0.] : 0.6875
[2, 16, 12] : [0.] : 0.75
[2, 16, 13] : [0.] : 0.8125
[2, 16, 14] : [0.] : 0.875
[2, 16, 15] : [0.] : 0.9375
[2, 16, 16] : [0.] : 1.0
[2, 16, 17] : [0.] : 1.0625
[2, 16, 18] : [0.] : 1.125
[2, 16, 19] : [0.] : 1.1875
[2, 17, 0] : [0.] : 0.0
[2, 17, 1] : [0.] : 0.058823529411764705
[2, 17, 2] : [0.] : 0.11764705882352941
[2, 17, 3] : [0.] : 0.17647058823529413
[2, 17, 4] : [0.] : 0.23529411764705882
[2, 17, 5] : [0.] : 0.29411764705882354
[2, 17, 6] : [0.] : 0.35294117647058826
[2, 17, 7] : [0.] : 0.4117647058823529
[2, 17, 8] : [0.] : 0.47058823529411764
[2, 17, 9] : [0.] : 0.5294117647058824
[2, 17, 10] : [0.] : 0.5882352941176471
[2, 1